# EXTRACT
# Extracting Women clothes from ASOS API (2)


(2) Extract more products similar to t-shirt category

### 0. Import modules

In [3]:
import requests
import time
import pandas as pd
import pickle

In [4]:
from logs.PASSES import RAPID ## my TOKEN

In [5]:
len(RAPID)

50

## RapidAPI - Asos

What is RapidAPI?
RapidAPI is an API Marketplace for developers to find, connect, and manage their API connections. 

https://rapidapi.com/apidojo/api/zappos1

In [7]:
headers = {
    "X-RapidAPI-Key": RAPID,
    "X-RapidAPI-Host": "asos2.p.rapidapi.com"
}

### Get countries available to get ASOS data from

Extract the relevant info for **SPAIN**

In [8]:
url = "https://asos2.p.rapidapi.com/countries/list"

response = requests.get(url, headers=headers)

countries_res=response.json()

In [ ]:
ES_index = [index for index, countries in enumerate(countries_res) if countries.get("country") == "ES"]
nombre_pais=countries_res[ES_index[0]]["name"]
pais=countries_res[ES_index[0]]["country"]
idioma=countries_res[ES_index[0]]["languages"][0]["language"]
moneda=countries_res[ES_index[0]]["currencies"][0]["currency"]

#display(nombre_pais, pais, idioma, moneda)

### 5 main brand categories
keep `WW_FP A-Z Women's Brands`

In [10]:
url = "https://asos2.p.rapidapi.com/categories/list"
querystring = {"country":"pais", "lang":"idioma"}
response = requests.get(url, headers=headers, params=querystring)
brands_res=response.json()

In [11]:
[print(R["alias"],R["content"]["title"]) for R in brands_res["brands"]] 
main_cats=brands_res["brands"][2] ## WW_FP A-Z Women's Brands

MW_FP A-Z Men's Brands
MW_MD A-Z Men's Outlet Brands
WW_FP A-Z Women's Brands
WW_MD A-Z of brands: Outlet & sale
WW_Beauty A-Z Face + Body Brands
5


### Brand ids in `WW_FP A-Z Women's Brands`

In [ ]:
[print(R["content"]["title"], R["link"]["categoryId"]) for R in main_cats["children"]]

#Roxy 7274
#Quiksilver 28829
#Lacoste 7294

### Get data directly from category id

In [15]:
url = "https://asos2.p.rapidapi.com/categories/list"

querystring = {"country":"pais", "lang":"idioma"}

response = requests.get(url, headers=headers, params=querystring)

res=response.json()

In [16]:
res["navigation"][0] #men
res["navigation"][1] #women
resWomen=res["navigation"][1]

### Create a list containing the type of items and their category id

In [18]:
category_list=[]
for RESW in resWomen["children"]:
   # display(RESW["children"])
    for i in range(len(RESW["children"])):
        RESW2=RESW["children"][i]["children"]
        for j in RESW2:
            RESW3=j["children"]
            for n in RESW3:
                #display(n["link"]["categoryId"])
                #display(n["link"]["webUrl"])
                category_list.append(n["link"]["webUrl"])

In [19]:
category_list2=[CAT.split("cid=") for CAT in category_list]
category_list2=[CAT for CAT in category_list2 if len(CAT)==2]


### CategoryId of Woman T-shirts is # 20581

In [ ]:
#find_category="t-shirt"
#find_category="top"
find_category="shirt"


category_listx=[]
for CAT in category_list2:
    if(find_category in CAT[0]):
        category_listx.append(CAT)
        
category_listx[:3]

### "shirts" category

In [ ]:
#find_category="t-shirt"
#find_category="top"
find_category="shirt"

category_listx=[]
for CAT in category_list2:
    if(find_category in CAT[0]):
        category_listx.append(CAT)
        
category_listx

In [78]:
categories=[VALS[1] for VALS in category_listx]
categories=list(set(categories))
categories_shirt=categories

['15200', '20581', '13053', '27423', '11321', '20580']

### "tops" category

In [130]:
#find_category="t-shirt"
find_category="top"
#find_category="shirt"

category_listx=[]
for CAT in category_list2:
    if(find_category in CAT[0]):
        category_listx.append(CAT)

category_listx=[VALS for VALS in category_listx if "brands" not in  VALS[0]]
category_listx=[VALS[1] for VALS in category_listx]
categories=[VALS.split("&")[0] for VALS in category_listx]
categories_top=categories
categories_top

['20581', '4167', '4167', '20581', '4169', '27167', '9581', '9581', '27421']

In [117]:
find_category="t-shirt"
#find_category="top"
#find_category="shirt"

category_listx=[]
for CAT in category_list2:
    if(find_category in CAT[0]):
        category_listx.append(CAT)
category_listx     

[['https://www.asos.com/us/women/sale/tops/t-shirts-tank-tops/cat/?', '20581'],
 ['https://www.asos.com/us/women/sale/tops/t-shirts-tank-tops/cat/?', '20581']]

In [99]:
categories=[VALS[1] for VALS in category_listx]
categories=list(set(categories))
categories=[VALS.split("&")[0] for VALS in categories]
categories_top=categories

### "dresses" category

In [ ]:
#find_category="t-shirt"
find_category="dresses"
#find_category="shirt"

category_listx=[]
for CAT in category_list2:
    if(find_category in CAT[0]):
        category_listx.append(CAT)

category_listx=[VALS for VALS in category_listx if "brands" not in  VALS[0]]
category_listx=[VALS[1] for VALS in category_listx]
categories=[VALS.split("&")[0] for VALS in category_listx]
categories_dress=categories


### Iterate the search to get all the products available

Asos has a limit of 48 products by request.

Change the category Id to extract the products of each category


In [143]:
url = "https://asos2.p.rapidapi.com/products/v2/list"
headers = {
    "X-RapidAPI-Key": RAPID,
    "X-RapidAPI-Host": "asos2.p.rapidapi.com"
}

# categories_dress
# "20581" #only t-shirt
#categories_top
#categories_shirt


category_id = categories_dress # "20581"
limit = 48
offset = 0
results = []

while True:
    querystring = {
        "categoryId": categories_dress, #"20581",
        "lang":{idioma},"currency":{moneda},"store":{pais},"country":{pais},
        "limit": str(limit),
        "offset": str(offset),
        "sort": "freshness"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()
        results.extend(data['products'])  # Append the products from the current page to results
        offset += limit  # Increment the offset to get the next page
        if len(data['products']) < limit:
            break  # Break the loop if the current page has fewer than the limit
    else:
        print(f"Error: {response.status_code}")
        break  # Exit the loop on error



### Parse json to DataFrame

In [ ]:
thedf=[[row["brandName"], row["name"],  row["price"]["current"]["text"], row["colour"]] for row in results]
tets_df=pd.DataFrame(thedf,columns=["brand", "description", "price", "colour"])
tets_df[:2]

### Save the data using pickle

The Lines are commented to avoid overwritting

In [ ]:
# asos_womendress_data=results #only t-shirts
## asos_womentshirt_data=results #only t-shirts
## asos_womenshirt2_data=results # all "shirts"
# asos_womentop_data=results # all "shirts"

In [146]:
### Save the object to a file using pickle
#with open('asos_womendress_data.pkl', 'wb') as file:
#    pickle.dump(asos_womendress_data, file)

### Save the object to a file using pickle
#with open('asos_womentop_data.pkl', 'wb') as file:
#    pickle.dump(asos_womentop_data, file)

### Save the object to a file using pickle
#with open('asos_womenshirt2_data.pkl', 'wb') as file:
#    pickle.dump(asos_womenshirt2_data, file)
    
### Save the object to a file using pickle
#with open('asos_womentshirt_data.pkl', 'wb') as file:
#    pickle.dump(asos_womentshirt_data, file)

In [36]:
with open('../../data/raw/asos_womentshirt_data.pkl', 'rb') as file:
    asos_womentshirt_data = pickle.load(file)

In [58]:
asos_raw=asos_womenshirt2_data

thedf2=[[row["brandName"], row["name"],  row["price"]["current"]["text"], row["colour"]] for row in asos_raw]
color_df2=pd.DataFrame(thedf2,columns=["brand", "description", "price", "colour"])
color_df2.tail()

,brand,description,price,colour
737,New Girl Order Tall,Sudadera de chándal con diseño tie dye de New ...,"8,50 €",Teñido efecto anudado multicolor
738,Topshop,Sudadera salvia limpia con capucha de Topshop,"32,25 €",Verde
739,New Look,Sudadera amarillo claro con texto estampado 'H...,"11,50 €",Amarillo claro
740,New Girl Order Petite,Sudadera corta azul efecto tie dye con capucha...,"24,00 €",Azul
741,New Girl Order Curve,Sudadera de chándal con diseño tie dye de New ...,"22,25 €",Violeta teñido
